In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome("chromedriver.exe")

WebDriverException: Message: unknown error: unrecognized Blink revision: 416d6d8013e9adb6dd33b0c12e7614ff403d1a94
  (Driver info: chromedriver=2.10.267521,platform=Windows NT 6.3 x86_64)


In [62]:
test = []
for div in coverpage_news:
    test.append(div.find('a')['href'])
    
    
test = pd.DataFrame(
    {'Article Link': test})

#len(test['Article Link'].unique())
test.tail()

,Article Link
2753,/Article/Brazil-corn-production-to-fall-5-soy-...
2754,/Article/US-corn-and-soy-harvest-rates-confoun...
2755,/Article/Speculators-increasingly-bearish-on-s...
2756,/Article/Saudi-Arabia-signs-MOU-to-invest-in-R...
2757,/Article/Vegetable-oil-prices-could-fall-8-on-...


In [65]:
import pdb

# Scraping the first 5 articles
number_of_articles = 5
# Empty lists for content, links and titles
news_contents = []
list_links = []
list_titles = []
news_keywords = []
news_published_time = []
source_orgs = []

for n in np.arange(0, number_of_articles):
    
    # only news articles (there are also albums and other things)
    #if "inenglish" not in coverpage_news[n].find('a')['href']:  
    #    continue
    
    # Getting the link of the article
    link = coverpage_news[n].find('a')['href']
    # Getting the title
    title = coverpage_news[n].get_text() #.find('a')
    list_titles.append(title)
    list_links.append(link)
   
    # Reading the content (it is divided in paragraphs)
    article = requests.get(link)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    article_keys = soup_article.find("meta", {"itemprop":"specialty"})['content']
    source_org = soup_article.find("meta", {"itemprop":"sourceOrganization"})['content']
    article_published_time = soup_article.find("meta", {"itemprop":"dateCreated"})['content']
    source_orgs.append(source_org)
    news_keywords.append(article_keys)
    news_published_time.append(article_published_time)
    body = soup_article.find_all('div', class_='group')
    #pdb.set_trace()
    #print(body)
    final_article = ""
    if len(body) == 0:
        body = soup_article.find("div", {"id": "video-meta-data"})#.find('p', id_='video-meta-data')
        body = [body]
        #pdb.set_trace()
 #final_article = ""
#        news_contents.append(final_article)
 #       continue
    #print(n, "article")
    #print(article_keys)
    #print(news_published_time)
    x = body[0].find_all('p')
    # Unifying the paragraphs
    list_paragraphs = []
    for p in np.arange(0, len(x)):
        #print(p)
        paragraph = x[p].get_text()
        list_paragraphs.append(paragraph)
        final_article = " ".join(list_paragraphs)
        
    news_contents.append(final_article)

In [66]:
# df_features
df_features = pd.DataFrame(
     {'Article Content': news_contents,
      'Article Title': list_titles,
      'Article Link': list_links,
      'news_keywords': news_keywords,
      'news_published_time': news_published_time,
      'source_site': source_orgs
    })

df_features.head()

,Article Content,Article Title,Article Link,news_keywords,news_published_time,source_site
0,"SINGAPORE, Oct 25 (Reuters) - Chicago soybean ...",GRAINS-Soybeans tick higher; dismal Chinese de...,https://www.cnbc.com/2019/10/24/reuters-americ...,"Agriculture,Soybeans,Australia,Chicago,United ...",2019-10-25T02:06:00+0000,CNBC
1,* Soybean futures have gained on talk of China...,GRAINS-Soybeans pare gains after surge on Chin...,https://www.cnbc.com/2019/10/23/reuters-americ...,"China,Donald Trump,Agriculture,Soybeans,Algeri...",2019-10-23T12:27:00+0000,CNBC
2,"purchases - sources@ (Adds lack of new buying,...",UPDATE 2-China offers tariff-free quota for 10...,https://www.cnbc.com/2019/10/22/reuters-americ...,"Donald Trump,Trade,Taxes,Soybeans,United State...",2019-10-22T22:29:00+0000,CNBC
3,* Beijing offers tariff waivers for U.S. soy i...,GRAINS-Soybeans firm on Chinese trade prospects,https://www.cnbc.com/2019/10/22/reuters-americ...,"China,Politics,Chicago,Illinois,Trade,Taxes,US...",2019-10-22T17:30:00+0000,CNBC
4,"purchases - sources@ (Releads, adds detail and...",UPDATE 1-China offers tariff-free quota for 10...,https://www.cnbc.com/2019/10/22/reuters-americ...,"Donald Trump,United States,Beijing",2019-10-22T12:55:00+0000,CNBC


In [62]:
df_features.tail(10)

,Article Content,Article Title,Article Link,news_keywords,news_published_time
40,"(Recasts with latest activity, updates prices,...","GRAINS-Corn, soybean futures slide on benefici...",https://www.cnbc.com/2019/08/19/reuters-americ...,"Donald Trump,United States,Chicago,Illinois,Io...",2019-08-19T17:50:00+0000
41,"third day@ (Recasts, updates with U.S. trading...",GRAINS-U.S. soy futures fall with exports in f...,https://www.cnbc.com/2019/08/14/reuters-americ...,"Asia: Markets,Credit Market,Donald Trump,Asia ...",2019-08-14T16:04:00+0000
42,* Chicago corn futures down 2.6% to weakest si...,GRAINS-Corn falls further after USDA raises cr...,https://www.cnbc.com/2019/08/13/reuters-americ...,"France,Chicago,Illinois,Futures & Commodities,...",2019-08-13T11:55:00+0000
43,"BEIJING, Aug 8 (Reuters) - China's soybean imp...",Chinaâ€™s July soybean imports rise on higher ...,https://www.cnbc.com/2019/08/07/reuters-americ...,"Politics,Trade,Soybeans,Environment,China",2019-08-08T03:35:00+0000
44,"BEIJING, Aug 8 (Reuters) - * China's soybean i...",China July soybean imports hit highest since A...,https://www.cnbc.com/2019/08/07/reuters-americ...,"Energy,Soybeans,Environment,China",2019-08-08T03:30:00+0000
45,"SYDNEY, Aug 8 (Reuters) - U.S. soybean futures...",GRAINS-Soybean prices edge down as U.S.-China ...,https://www.cnbc.com/2019/08/07/reuters-americ...,"China,Politics,Washington DC,Trade,US: News,Fu...",2019-08-08T01:56:00+0000
46,"* Market weighs Midwest weather, awaits USDA A...","GRAINS-Corn, soy steady as crop outlook weighe...",https://www.cnbc.com/2019/08/07/reuters-americ...,"France,Weather & Natural Disasters,Futures & C...",2019-08-07T10:53:00+0000
47,halts U.S. ag purchases@ * Excess moisture in ...,GRAINS-Corn futures rally on U.S. weather conc...,https://www.cnbc.com/2019/08/05/reuters-americ...,"Donald Trump,Futures & Commodities,Agriculture...",2019-08-05T19:19:00+0000
48,* Soybeans fall on escalating Washington-Beiji...,GRAINS-Soybeans extend decline on U.S.-China t...,https://www.cnbc.com/2019/08/05/reuters-americ...,"China,Politics,Donald Trump,Washington DC,Trad...",2019-08-05T10:12:00+0000
49,game: Russell@ (The opinions expressed here ar...,"COLUMN-For China, U.S. soybeans are small fry;...",https://www.cnbc.com/2019/07/30/reuters-americ...,"China,Donald Trump,Shanghai,US Energy Corp",2019-07-30T06:24:00+0000
